In [7]:
from Datos import Datos
from Arm import ArmkNN
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)


listUsers = np.unique(kNNdataset[:,0])
knn = ArmkNN(20)

In [2]:
knn.rec_item(kNNdataset,1)

3275.0

In [ ]:
# Prueba de kNN
train, test = train_test_split(kNNdataset, train_size=0.1)

num = 0
den = 0

for u in listUsers:
    item = knn.rec_item(train, u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        train = np.vstack((train,hit))
        den += 1
        if(hit[0,2] >= 3):
            num += 1
        print(num/den)
    else:
        train = np.vstack((train,[u,item,1.]))
        print('fail')

1.0
fail
fail
fail


Pruebas de NB

In [1]:
from Datos import Datos
from Arm import ArmNB
import numpy as np

from sklearn.model_selection import train_test_split


dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)

train, test = train_test_split(kNNdataset, train_size=0.1)

arm = ArmNB(train)

arm.rec_item(1)

898.0

In [2]:
from Datos import Datos
from Arm import ArmNB
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)


listUsers = np.unique(kNNdataset[:,0])
train, test = train_test_split(kNNdataset, train_size=0.1)
arm = ArmNB(train)

In [3]:
# Prueba de NB


num = 0
den = 0

for u in listUsers:
    item = arm.rec_item(u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        train = np.vstack((train,hit))
        den += 1
        if(hit[0,2] >= 3):
            num += 1
        print(num/den)
    else:
        train = np.vstack((train,[u,item,1.]))

fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
1.0
fail
fail
fail
fail
fail
0.5
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
0.6666666666666666
fail
fail
0.75
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
0.6
0.5
fail
fail
fail
0.5714285714285714
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
0.625
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
0.6666666666666666
0.7
fail
fail
fail
0.7272727272727273
fail
fail
fail
fail
fail
fail
0.75
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
0.6923076923076923
fail
fail
fail
fail
fail
fail
fail
0.7142857142857143
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
0.7333333333333333
fail
fail
fail
fail
0